In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_json,col
from pyspark.sql.types import *
from os.path import abspath

spark = SparkSession\
        .builder\
        .appName("pyspark-notebook")\
        .master("spark://spark-master:7077")\
        .config("spark.executor.memory", "512m")\
        .config("hive.metastore.uris", "thrift://hive-metastore:9083")\
        .config("spark.sql.warehouse.dir", "/user/hive/warehouse")\
        .enableHiveSupport()\
        .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

23/04/02 17:57:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
!pip install pandas
!pip install xlrd


import requests
import pandas as pd
from datetime import datetime

     |████████████████████████████████| 11.3 MB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 15.7 MB 2.8 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
     |████████████████████████████████| 96 kB 1.6 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
def get_aemet_data():
    url = "https://www.aemet.es/es/eltiempo/observacion/ultimosdatos_C447A_datos-horarios.xls?k=coo&l=C447A&datos=det&w=0&f=temperatura&x="
    r = requests.get(url)
    df = pd.read_excel(r.content, skiprows=3)
    return df

In [4]:
datos = get_aemet_data()
datos = datos.rename(columns={"Fecha y hora oficial": 'Fecha', "Temperatura (ºC)" : 'Temperatura_c',
                             "Velocidad del viento (km/h)": 'Velocidad_del_viento_km_h', "Dirección del viento": 'Direccion_del_viento',
                             "Racha (km/h)": 'Racha_km_h', "Dirección de racha":'Direccion_de_racha',
                             "Precipitación (mm)": 'Precipitacion_mm', "Presión (hPa)":'Presion_hPa',
                             "Tendencia (hPa)": 'Tendencia_hPa', "Humedad (%)": 'Humedad_percent'})


In [5]:
for index, row in datos.iterrows():
    fecha_no_f = row['Fecha']
    try:
        fecha_objeto = datetime.strptime(fecha_no_f, '%Y-%m-%d %H:%M:%S')
        datos.iloc[index, datos.columns.get_loc('Fecha')] = fecha_objeto.strftime('%d/%m/%Y %H:%M')
    except ValueError:
        print(f'La fecha {fecha_no_f} no tiene el formato correcto')


La fecha 02/04/2023 18:00 no tiene el formato correcto
La fecha 02/04/2023 17:00 no tiene el formato correcto
La fecha 02/04/2023 16:00 no tiene el formato correcto
La fecha 02/04/2023 15:00 no tiene el formato correcto
La fecha 02/04/2023 14:00 no tiene el formato correcto
La fecha 02/04/2023 13:00 no tiene el formato correcto
La fecha 02/04/2023 12:00 no tiene el formato correcto
La fecha 02/04/2023 11:00 no tiene el formato correcto
La fecha 02/04/2023 10:00 no tiene el formato correcto
La fecha 02/04/2023 09:00 no tiene el formato correcto
La fecha 02/04/2023 08:00 no tiene el formato correcto
La fecha 02/04/2023 07:00 no tiene el formato correcto
La fecha 02/04/2023 06:00 no tiene el formato correcto
La fecha 02/04/2023 05:00 no tiene el formato correcto
La fecha 02/04/2023 04:00 no tiene el formato correcto
La fecha 02/04/2023 03:00 no tiene el formato correcto
La fecha 02/04/2023 02:00 no tiene el formato correcto
La fecha 02/04/2023 01:00 no tiene el formato correcto
La fecha 0

In [9]:
schema = StructType([
    StructField('Fecha', TimestampType(), True),
    StructField('Temperatura_c', DoubleType(), True),
    StructField('Velocidad_del_viento_km_h', DoubleType(), True),
    StructField('Direccion_del_viento', StringType(), True),
    StructField('Racha_km_h', DoubleType(), True),
    StructField('Direccion_de_racha', StringType(), True),
    StructField('Precipitacion_mm', DoubleType(), True),
    StructField('Presion_hPa', DoubleType(), True),
    StructField('Tendencia_hPa', DoubleType(), True),
    StructField('Humedad_percent', DoubleType(), True)
    
])
spark_df = spark.createDataFrame(datos)
spark_df.show(3)

+----------------+-------------+-------------------------+--------------------+----------+------------------+----------------+-----------+-------------+---------------+
|           Fecha|Temperatura_c|Velocidad_del_viento_km_h|Direccion_del_viento|Racha_km_h|Direccion_de_racha|Precipitacion_mm|Presion_hPa|Tendencia_hPa|Humedad_percent|
+----------------+-------------+-------------------------+--------------------+----------+------------------+----------------+-----------+-------------+---------------+
|02/04/2023 18:00|         22.4|                       12|                Este|        21|              Este|               0|      948.6|         -1.2|             47|
|02/04/2023 17:00|         23.7|                       11|                Este|        21|           Sudeste|               0|      948.8|         -1.6|             37|
|02/04/2023 16:00|         22.8|                       13|             Sudeste|        24|              Este|               0|      949.2|         -1.9|   

## 3. Carga de los datos a la tabla en hive

In [10]:
DATA_BASES = spark.sql("SHOW DATABASES ")


In [11]:
DATA_BASES.show()

+-------------------+
|          namespace|
+-------------------+
|            default|
|        open_datadb|
|open_weather_datadb|
+-------------------+



In [12]:
spark.sql("use open_weather_datadb")

DataFrame[]

In [13]:
spark_df.write.mode('overwrite').saveAsTable('weather')

In [14]:
a = spark.sql("select * from weather")

In [15]:
a.show(3)

+----------------+-------------+-------------------------+--------------------+----------+------------------+----------------+-----------+-------------+---------------+
|           Fecha|Temperatura_c|Velocidad_del_viento_km_h|Direccion_del_viento|Racha_km_h|Direccion_de_racha|Precipitacion_mm|Presion_hPa|Tendencia_hPa|Humedad_percent|
+----------------+-------------+-------------------------+--------------------+----------+------------------+----------------+-----------+-------------+---------------+
|02/04/2023 06:00|         17.4|                       17|                 Sur|        30|           Sudeste|               0|      950.0|         -1.1|             40|
|02/04/2023 05:00|         17.4|                       17|                 Sur|        30|           Sudeste|               0|      950.2|         -1.5|             36|
|02/04/2023 04:00|         18.1|                       18|             Sudeste|        26|               Sur|               0|      950.6|         -1.7|   